In [ ]:
import os
import cv2
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
HEIGHT = 480
WIDTH = 480
CHANNELS = 3
CLASSES = 6
top_dropout_rate = 0.2

In [ ]:
weights_path = "../input/efnet-v2-insectaug-model/EfficientNet_V2_InsectAug.h5"
hub_url = '../input/efficientnetv2-tfhub-weight-files/tfhub_models/efficientnetv2-l-21k-ft1k/feature_vector'

In [ ]:
def get_model():
    model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape = [HEIGHT, WIDTH, CHANNELS]),
            hub.KerasLayer(hub_url, trainable = True),
            Dropout(top_dropout_rate, name = "top_dropout"),
            Dense(CLASSES, activation = 'sigmoid')
        ])
    model.build((HEIGHT, WIDTH, CHANNELS))
    model.compile(optimizer ='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.summary()
    return model

In [ ]:
model = get_model()
model.load_weights(weights_path)

In [ ]:
test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission

In [ ]:
def load_image(image_id):
    file_path = str(image_id)
    img = cv2.imread(test_img+'/'+file_path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
def insect_augmentation(image, n_insects = 2, dark_insect=False, p = 0.5, insects_folder='../input/insect/insect'):
    aug_prob = random.random()
    flag = False
    if aug_prob < p:
        flag = True
        height, width, _ = image.shape  # target image width and height
        insects_images = os.listdir(insects_folder)
        img_shape = image.shape

        for _ in range(n_insects):
            insect = cv2.cvtColor(cv2.imread(os.path.join(insects_folder, random.choice(insects_images))), cv2.COLOR_RGB2BGR)
            insect = cv2.flip(insect, random.choice([-1, 0, 1]))
            insect = cv2.rotate(insect, random.choice([0, 1, 2]))
            insect = cv2.resize(insect, (width, height))

            h_height, h_width, _ = insect.shape  # insect image width and height
            roi_ho = random.randint(0, image.shape[0] - insect.shape[0])
            roi_wo = random.randint(0, image.shape[1] - insect.shape[1])
            roi = image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            # Creating a mask and inverse mask 
            img2gray = cv2.cvtColor(insect, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            #mask_inv = cv2.cvtColor(cv2.bitwise_not(mask),cv2.COLOR_BGR2GRAY)
            mask_inv = cv2.bitwise_not(mask)

            # Now black-out the area of insect in ROI
            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

            # Take only region of insect from insect image.
            if dark_insect:
                img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                insect_fg = cv2.bitwise_and(img_bg, img_bg, mask=mask)
            else:
                insect_fg = cv2.bitwise_and(insect, insect, mask=mask)

            # Put insect in ROI and modify the target image
            dst = cv2.add(img_bg, insect_fg, dtype = cv2.CV_64F)

            image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst


    return image

In [ ]:
def process(img):
    img = insect_augmentation(img, n_insects = random.randint(1, 2), dark_insect=False, p = 0.2)
    img = tf.cast(img, tf.float32) / 255.0
    img = np.array(img)
    return cv2.resize(img , (480, 480)).reshape(-1, 480, 480, 3)
def predict(img):
    img = load_image(img)
    tta_steps = 5
    predictions = []
    for i in range(tta_steps):
        pred = model.layers[2](model.layers[1](model.layers[0](process(img)))).numpy()[0]
        predictions.append(pred)
    result = np.median(predictions, axis=0)
    
    return result

In [ ]:
test_images = submission['image'][0]
test_images = load_image(test_images)
preds = model.layers[2](model.layers[1](model.layers[0](process(test_images)))).numpy()[0]
print(preds)

In [ ]:
n_label = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
label = []
for i in range(len(submission['image'])):
    test_images = submission['image'][i]
    preds = predict(test_images)
    answer = []
    for j in range(len(preds)):
        if preds[j] > 0.2:
            answer.append(n_label[j])
    answer = ' '.join(answer)
    label.append(answer)
submission['labels'] = label
submission

In [ ]:
submission.to_csv('submission.csv', index=False)